In [27]:
!pip3 install transformers
!pip3 install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [3]:
!pip3 install --upgrade transformers

In [4]:
import transformers
from transformers import BertTokenizer, BertForMaskedLM
import torch
import random
from tqdm.auto import tqdm
from torch.optim import AdamW

In [5]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
data_path="/content/fire_flower.txt"

In [9]:
with open(data_path,'r') as f:
  data=f.read().split('/n')

In [10]:
for sentence in data:
  if len(sentence)<50:
    data.remove(sentence)

In [11]:
print(len(data))

1


In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
inputs=tokenizer(data,max_length=512,padding=True,return_tensors='pt')
inputs.keys()

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2752: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


KeysView({'input_ids': tensor([[  101,  1037,  1000,  2543,  6546,  1000,  2064,  6523,  2000,  1037,
          2373,  1011,  2039,  1999, 10022,  1005,  1055,  3565,  7986, 10243,
          1012,  2186,  2008,  8624,  7986,  1996,  3754,  2000,  5466,  2543,
         18510,  1010,  2030,  2009,  2064,  2022,  1037,  2691,  2171,  2005,
          2536,  2613,  1011,  2088,  4264,  2008, 13014,  7311,  1010,  2107,
          2004,  1996,  1043, 10626, 10735,  2050,  7094,  2030,  2543, 18041,
          1012,  1999,  2678,  2399,  3565,  7986,  2186,  1024,  1996,  2543,
          6546,  2003,  1037,  2373,  1011,  2039,  2008,  4473,  7986,  2000,
          5607,  2543, 18510,  2000,  4154,  6716,  1998,  3338,  5991,  1012,
          2043,  7986, 17427,  1037,  2543,  6546,  1010,  2002, 21743,  2046,
          2010,  2543,  1011,  6113,  1000,  2543,  7986,  1000,  2433,  1012,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
inputs['labels']=inputs['input_ids'].detach().clone()


In [15]:
random_tensor=torch.rand(inputs['input_ids'].shape)

In [16]:
masked_tensors=(random_tensor<0.15)*(inputs['input_ids']!='101')*(inputs['input_ids']!='102')*(inputs['input_ids']!='0')

In [17]:
nonzero_indices=[]
for i in range(len(masked_tensors)):
  nonzero_indices.append(torch.flatten(masked_tensors[i].nonzero()).tolist())

In [18]:
for i in range(len(inputs['input_ids'])):
  inputs['input_ids'][i,nonzero_indices[i]]=103

In [19]:
inputs['input_ids']

tensor([[  103,  1037,  1000,  2543,  6546,  1000,  2064,  6523,  2000,  1037,
           103,   103,  2039,  1999, 10022,  1005,  1055,  3565,  7986, 10243,
           103,  2186,  2008,  8624,  7986,   103,  3754,   103,  5466,  2543,
           103,  1010,  2030,  2009,  2064,  2022,  1037,  2691,  2171,  2005,
          2536,  2613,  1011,  2088,  4264,  2008, 13014,  7311,  1010,  2107,
          2004,   103,  1043, 10626, 10735,  2050,  7094,  2030,  2543, 18041,
          1012,  1999,  2678,  2399,  3565,  7986,  2186,  1024,  1996,  2543,
          6546,  2003,   103,  2373,  1011,  2039,  2008,  4473,  7986,  2000,
          5607,  2543,   103,  2000,  4154,  6716,  1998,  3338,  5991,  1012,
           103,   103, 17427,  1037,  2543,  6546,  1010,  2002, 21743,  2046,
          2010,  2543,   103,  6113,  1000,  2543,  7986,  1000,   103,   103,
           102]])

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [21]:
class BookDataset(torch.utils.data.Dataset):
  def __init__(self,encodings):
    self.encoding=encodings
  def __len__(self):
    return len(self.encoding)
  def __getitem__(self,index):
    input_ids=self.encodings['input_ids']
    attention_mask=self.encodings['attention_mask']
    token_type_ids=self.encodings['token_type_ids']
    labels=self.encodings['labels']
    return {'input_ids':input_ids,'attention_mask':attention_mask,'token_type':token_type_ids,'labels':labels}



In [22]:
model=BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
dataset=BookDataset(inputs)
dataloader=torch.utils.data.DataLoader(data,batch_size=16,shuffle=True)
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
## D

In [24]:
epochs=2
optimizer=AdamW(model.parameters(),lr=5e-5)
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [25]:
for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        optimizer.zero_grad()

        # Assume batch is a list of strings
        encoding = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        labels = input_ids.clone()  # for LM, labels = input_ids

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
## second example

In [32]:
from datasets import DatasetDict,Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling,AutoModelForSequenceClassification
import evaluate
import numpy as np
from datasets import load_dataset

from transformers import DataCollatorWithPadding

In [33]:
dataset_dict=load_dataset("shawhin/phishing-site-classification")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
## Loading pretrained Model

In [34]:
model_path="google/bert_uncased_L-2_H-128_A-2"
## Load model tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [35]:
## load model with binary classification
id2label={0:"safe",1:"Not Safe"}
label2id={"safe":0,"Not Safe":1}

In [38]:
model=AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2,id2label=id2label,label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## SET TRAINABLE PARAMETERS

In [39]:
for name,param in model.named_parameters():
  param.requires_grad=False
#unfreeze base model pooling layers
for name,param in  model.base_model.named_parameters():
  if "pooler" in name:
    param.requires_grad=True

In [ ]:
## Data Preprocessing

In [40]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True)
##preprocess all datasets
tokenized_datasets=dataset_dict.map(preprocess_function,batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
## Evaluation metrics

In [41]:
accuracy=evaluate.load("accuracy")
auc_score=evaluate.load("roc_auc")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Convert logits to probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)
    positive_class_prob = probabilities[:, 1]

    # Compute metrics
    auc_val = auc_score.compute(
        references=labels,
        prediction_scores=positive_class_prob
    )["roc_auc"]

    acc_val = accuracy.compute(
        predictions=np.argmax(predictions, axis=1),
        references=labels
    )["accuracy"]

    return {
        "accuracy": round(acc_val, 4),
        "auc": round(auc_val, 4)
    }


In [42]:
 from transformers import TrainingArguments, Trainer

In [ ]:
## hyperparameters

In [43]:
lr=2e-4
batch_size=4
num_epochs=10
training_args = TrainingArguments(
    output_dir="test_trainer",
    learning_rate=2e-4,
    eval_strategy="epoch",       # run evaluation each epoch
    save_strategy="epoch",       # save checkpoints each epoch
    logging_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    load_best_model_at_end=True  # now compatible
)

In [44]:
## FineTune
trainer=Trainer(model=model,args=training_args,train_dataset=tokenized_datasets["train"],eval_dataset=tokenized_datasets["test"],compute_metrics=compute_metrics)

In [45]:
from transformers import DataCollatorWithPadding

# Create a data collator that pads sequences dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,  # <- this fixes the length mismatch
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-2521256194.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saisowhitp (saisowhitp-no) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.640800,0.589444,0.742200,0.824700
2,0.564800,0.546961,0.740000,0.834700
3,0.535400,0.520589,0.771100,0.842100
4,0.520200,0.506912,0.773300,0.848500
5,0.496700,0.498135,0.764400,0.853800
6,0.495500,0.490106,0.766700,0.856500
7,0.491200,0.489786,0.777800,0.859200
8,0.487400,0.483259,0.773300,0.860200
9,0.485900,0.481813,0.775600,0.861200
10,0.491200,0.481149,0.773300,0.861800


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=5250, training_loss=0.5208907528831845, metrics={'train_runtime': 132.6664, 'train_samples_per_second': 158.292, 'train_steps_per_second': 39.573, 'total_flos': 2518800110880.0, 'train_loss': 0.5208907528831845, 'epoch': 10.0})

In [46]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.484600,0.471744,0.780000,0.867600
2,0.476300,0.468109,0.795600,0.870700
3,0.473000,0.458773,0.782200,0.872700
4,0.463400,0.455799,0.791100,0.873500
5,0.468800,0.456155,0.795600,0.873400
6,0.462500,0.454008,0.795600,0.873800
7,0.455900,0.457169,0.795600,0.874400
8,0.455500,0.453019,0.793300,0.874000
9,0.451600,0.452037,0.793300,0.874400
10,0.462200,0.451746,0.793300,0.874400


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=5250, training_loss=0.46537208193824403, metrics={'train_runtime': 95.9173, 'train_samples_per_second': 218.939, 'train_steps_per_second': 54.735, 'total_flos': 2518800110880.0, 'train_loss': 0.46537208193824403, 'epoch': 10.0})

In [ ]:
## validation

In [47]:
predictions=trainer.predict(tokenized_datasets["validation"])
## extract the logits and labels from the prediction object
logits=predictions.predictions
lables=predictions.label_ids
## use our compute metrics function
metrics=compute_metrics((logits,lables))
print(metrics)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'accuracy': 0.8378, 'auc': np.float64(0.9082)}
